# Lib

In [1]:
import boto3
import json
import os

In [2]:
env_vars = !cat .env

for var in env_vars:
    key, value = var.split("=", 1)
    os.environ[key] = value.replace('"', "")


# Cria o cliente Bedrock
client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

# Embedding logic

In [3]:
def generate_embedding(text):
    response = client.invoke_model(
        body=json.dumps({
            "inputText": text
        }),
        modelId='amazon.titan-embed-text-v1',
        contentType='application/json',
        accept='application/json',
    )


    response_body = json.loads(response['body'].read())

    return response_body['embedding']


# Categories

In [4]:
category_descriptions = {
    "esportes": "Roupas confortaveis e respiráveis para atividades físicas.",
    "formal": "Vestuário elegante e profissional para ambientes de trabalho.",
    "casual": "Roupas descontraídas e confortáveis para o dia a dia.",
    "festa": "Vestuário sofisticado e glamouroso para eventos especiais",
    "Frio": "Roupas quentes e aconchegantes para temperaturas baixas.",
}

category_embeddings = {}
for category, description in category_descriptions.items():
    embedding = generate_embedding(description)
    category_embeddings[category] = embedding

# classifier

In [11]:
!pip install -q scipy


[notice] A new release of pip is available: 23.2.1 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [5]:
# Cosine
from scipy.spatial.distance import cosine

def classify_product(product_description):
    product_embedding = generate_embedding(product_description)
    # quanto amias prox de 1, mais similar é o produto com a cateogria
    similarities = {
        category: 1 - cosine(product_embedding, category_embedding)
        for category, category_embedding in category_embeddings.items()
    }
    return max(similarities, key=similarities.get)

In [7]:
print('Entre com a descrição do produto')
product_description = input()
print("usuário:",product_description)
categoria = classify_product(product_description)
print(categoria)


Entre com a descrição do produto
usuário: Calça de frio
Frio


Para comparar o quanto dois itens, como descrições de sintomas ou categorias de produtos, são similares, utilizamos métricas de similaridade entre esses vetores.

Uma das métricas mais comuns é a distância cosseno, que mede o ângulo entre dois vetores no espaço. Se dois vetores estiverem apontando para direções semelhantes, o valor da distância cosseno será pequeno, indicando alta similaridade.

Dessa forma, a fórmula da similaridade cosseno é:

(A,B) = 1 − (A⋅B/ ∥A∥ ∥B∥)

